In [ ]:
!pip install lark graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
from lark import Lark, Transformer, v_args, Tree
from graphviz import Digraph

# Translated EBNF rules in .lark format
grammar = """
start: (annotation | vocabulary_box | extends_clause | concept_clause | relation_clause | aspect_clause | scalar_clause)*

annotation: "@" annotation_property ":"? type? STRING

annotation_property: ID  // E.g., dc, rdfs
ID: /[a-zA-Z_][a-zA-Z_0-9]*/  // Match identifiers
STRING: /"[^"]*"/  // Match double-quoted strings

vocabulary_box: "vocabulary" URI "as" ID "{" (extends_clause | concept_clause | relation_clause | aspect_clause | scalar_clause)* "}"

extends_clause: "extends" URI "as" ID

concept_clause: annotation* "@rdfs:comment" STRING "concept" ID concept_extension?

concept_extension: "<" ID "["?

relation_clause: annotation* "@rdfs:comment" STRING "relation" "entity" ID relation_body
relation_body: "[" relation_details "]"

relation_details: "from" ID "to" ID relation_flags*
relation_flags: "forward" ID | "reverse" ID | "transitive"

aspect_clause: annotation* "@rdfs:comment" STRING "aspect" ID aspect_details
aspect_details: "[" key_clause "]"
key_clause: "key" ID

scalar_clause: annotation* "@rdfs:comment" STRING "scalar" "property"? ID scalar_details
scalar_details: "[" domain_clause range_clause scalar_flags* "]" | "[" one_of_clause "]"

domain_clause: "domain" ID
range_clause: "range" ID ":"? ID
scalar_flags: "functional" | "transitive"
one_of_clause: "oneOf" value_list
value_list: STRING ("," STRING)*

concept: STRING
type: "comment" | "description"

URI: /<[^>]+>/  // Match URIs (e.g., <http://example.com>)

PREFID: /[a-zA-Z_][a-zA-Z_0-9]*:[a-zA-Z_0-9]*/

COMMENT: /\/\/[^\n]*/x  // Matches `//` followed by anything except a newline
%ignore COMMENT          // Ignore comments during parsing

%import common.ESCAPED_STRING
%import common.WS
%ignore WS
"""

In [ ]:
grammar = """
#   Copyright 2019 California Institute of Technology ("Caltech").
#   U.S. Government sponsorship acknowledged.

#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at

#  	  http://www.apache.org/licenses/LICENSE-2.0

#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

# Lark grammar equivalent of io.opencaesar.oml.dsl.Oml

# Hidden tokens (whitespace and comments)
WS: /[ \t\r\n]+/  -> skip
ML_COMMENT: /\/\*[\s\S]*?\*\//  -> skip
SL_COMMENT: /\/\/[^\n]*/  -> skip

# Imports (Lark does not have a direct import mechanism like Xtext, but you can include other grammars)
# You may need to handle namespace resolution manually if required.
# Reference to "http://opencaesar.io/oml" and "http://www.eclipse.org/emf/2002/Ecore" should be handled in preprocessing or semantics.

?ontology: vocabulary_box | description_box

annotation: "@" annotation_property [" " annotation_value ("," annotation_value)*]

annotation_value: literal_value | referenced_value

vocabulary_box: vocabulary | vocabulary_bundle

vocabulary: annotation* "vocabulary" namespace "as" prefix "{" (extension | usage)* vocabulary_statement* "}"

vocabulary_bundle: annotation* "vocabulary" "bundle" namespace "as" prefix "{" (extension | inclusion)* "}"

description_box: description | description_bundle

description: annotation* "description" namespace "as" prefix "{" (extension | usage)* description_statement* "}"

description_bundle: annotation* "description" "bundle" namespace "as" prefix "{" (extension | usage | inclusion)* "}"

specializable_term: type | annotation_property | scalar_property | unreified_relation

type: entity | scalar

entity: aspect | concept | relation_entity

aspect: annotation* ("aspect" name | "ref" "aspect" ref) ("[" key_axiom* "]")? entity_specialization? entity_equivalence?

concept: annotation* ("concept" name | "ref" "concept" ref) ("[" instance_enumeration_axiom? key_axiom* "]")? entity_specialization? entity_equivalence?

relation_entity: annotation* ("relation" "entity" name | "ref" "relation" "entity" ref) ("[" ("from" sources ("," sources)* )? ("to" targets ("," targets)* )? forward_relation? reverse_relation?(functional? "functional")? (inverse_functional? "inverse" "functional")? (symmetric? "symmetric")? (asymmetric? "asymmetric")? (reflexive? "reflexive")? (irreflexive? "irreflexive")? (transitive? "transitive")? key_axiom* "]")? entity_specialization? entity_equivalence?

entity_specialization: "<" specialization_axiom ("," specialization_axiom)* | specialization_axiom ("," specialization_axiom)*? "[" property_restriction_axiom* "]"

entity_equivalence: "=" entity_equivalence_axiom ("," entity_equivalence_axiom)*

entity_equivalence_axiom: super_entities ("&" super_entities)* | super_entities? "[" property_restriction_axiom* "]"

scalar: annotation* ("scalar" name | "ref" "scalar" ref) ("[" literal_enumeration_axiom "]")? scalar_specialization? scalar_equivalence?

scalar_specialization: "<" specialization_axiom ("," specialization_axiom)*

scalar_equivalence: "=" scalar_equivalence_axiom ("," scalar_equivalence_axiom)*

scalar_equivalence_axiom: super_scalar ("[" ("length" length? unsigned_integer)?
    ("minLength" min_length? unsigned_integer)?
    ("maxLength" max_length? unsigned_integer)?
    ("pattern" pattern? STRING)?
    ("language" language? ID)?
    ("minInclusive" min_inclusive? literal)?
    ("minExclusive" min_exclusive? literal)?
    ("maxInclusive" max_inclusive? literal)?
    ("maxExclusive" max_exclusive? literal)? "]")?


property: annotation_property | semantic_property

annotation_property: annotation* ("annotation" "property" name | "ref" "annotation" "property" ref) property_specialization? property_equivalence?

semantic_property: scalar_property | relation

scalar_property: annotation* ("scalar" "property" name | "ref" "scalar" "property" ref) ("[" ("domain" domains ("," domains)*)? ("range" ranges ("," ranges)*)? ("functional" "functional")? "]")? property_specialization? property_equivalence?

relation: forward_relation | reverse_relation | unreified_relation

forward_relation: annotation* "forward" name

reverse_relation: annotation* "reverse" name

unreified_relation: annotation* ("relation" name | "ref" "relation" ref) ("[" ("from" sources ("," sources)*)? ("to" targets ("," targets)*)? reverse_relation?(functional? "functional")? (inverse_functional? "inverse" "functional")? (symmetric? "symmetric")? (asymmetric? "asymmetric")? (reflexive? "reflexive")?(irreflexive? "irreflexive")? (transitive? "transitive")? "]")? property_specialization? property_equivalence?

property_specialization: "<" specialization_axiom ("," specialization_axiom)*

property_equivalence: "=" property_equivalence_axiom ("," property_equivalence_axiom)*

property_equivalence_axiom: super_property

property: annotation* ("rule" name | "ref" "rule" ref) ("[" antecedent ("&" antecedent)* "->" consequent ("&" consequent)* "]")?

builtin: annotation* ("builtin" name | "ref" "builtin" ref)

anonymous_instance: anonymous_concept_instance | anonymous_relation_instance

anonymous_concept_instance: "{" ":" concept_ref? "[" property_value_assertion* "]"

anonymous_relation_instance: target named_instance_ref "[" property_value_assertion* "]"

named_instance: concept_instance | relation_instance

concept_instance: annotation* ("instance" name | "ref" "instance" ref) (":" concept_type_assertion ("," concept_type_assertion)*)? "[" property_value_assertion* "]"

relation_instance: annotation* ("relation" "instance" name | "ref" "relation" "instance" ref) (":" relation_type_assertion ("," relation_type_assertion)*)?
    ("[" ("from" sources ("," sources)*)? ("to" targets ("," targets)*)? property_value_assertion* "]")?

vocabulary_statement: rule | builtin | specializable_term

description_statement: named_instance

import: extension | usage | inclusion

extension: kind "Extends" namespace ("as" prefix)?

usage: kind "Uses" namespace ("as" prefix)?

inclusion: kind "Includes" namespace ("as" prefix)?

specialization_axiom: super_term

property_restriction_axiom: property_self_restriction_axiom | property_range_restriction_axiom | property_cardinality_restriction_axiom | property_value_restriction_axiom

property_range_restriction_axiom: "restricts" kind "to" range

property_cardinality_restriction_axiom: "restricts" property "to" kind cardinality (range_clause)?

property_value_restriction_axiom: "restricts" property "to" (literal_value | contained_value | referenced_value)

property_self_restriction_axiom: "restricts" property "to" "self"

key_axiom: "key" properties ("," properties)*

instance_enumeration_axiom: "oneOf" instances ("," instances)*

literal_enumeration_axiom: "oneOf" literals ("," literals)*

concept_type_assertion: "type" concept_ref

relation_type_assertion: "type" relation_entity_ref

property_value_assertion: property property_value ("," property_value)*

property_value: literal_value | contained_value | referenced_value

predicate: unary_predicate | binary_predicate | built_in_predicate

unary_predicate: type_predicate | relation_entity_predicate

binary_predicate: property_predicate | same_as_predicate | different_from_predicate

type_predicate: type "(" argument ")"

relation_entity_predicate: type "(" argument1 "," argument "," argument2 ")"

property_predicate: property "(" argument1 "," argument2 ")"

same_as_predicate: "sameAs" "(" argument1 "," argument2 ")"

different_from_predicate: "differentFrom" "(" argument1 "," argument2 ")"

built_in_predicate: "builtIn" "(" built_in ref "," arguments ("," arguments)* ")"

argument: variable | literal | instance

literal: integer_literal | decimal_literal | double_literal | boolean_literal | quoted_literal

integer_literal: value

decimal_literal: value

double_literal: value

boolean_literal: value

quoted_literal: value ("^^" type | "$" lang_tag)?

range_restriction_kind: "all" | "some"

cardinality_restriction_kind: "exactly" | "min" | "max"

import_kind: "extends" -> extension | "uses" -> usage | "includes" -> inclusion

ref: id | qname | iri

cross_ref: qname | iri

boolean: BOOLEAN_STR

unsigned_integer: UNSIGNED_INTEGER_STR

integer: UNSIGNED_INTEGER_STR | INTEGER_STR

decimal: DECIMAL_STR

double: DOUBLE_STR

BOOLEAN_STR: "false" | "true"

UNSIGNED_INTEGER_STR: /[0-9]+/

INTEGER_STR: ("+" | "-")? /[0-9]+/

DECIMAL_STR: ("+" | "-")? (/[0-9]+(\.[0-9]*)?/ | /\.[0-9]+/)

DOUBLE_STR: ("+" | "-")? (/[0-9]+(\.[0-9]*)?/ | /\.[0-9]+/) ("e" | "E") ("+" | "-")? /[0-9]+/

STRING: /"{3}([^\\]|\\.)*"{3}|'{3}([^\\]|\\.)*'{3}|"([^"\\]|\\.)*"|'([^'\\]|\\.)*'/

NAMESPACE: /<([^>'\s#])*([#/])>/

IRI: /<[^>\s]*>/

ID: IDFRAG

QNAME: IDFRAG ":" IDFRAG

ML_COMMENT: /\/\*.*?\*\//

SL_COMMENT: /\/\/[^\n\r]*\r?\n?/

WS: /[ \t\r\n]+/

IDFRAG: "^"? (ALPHA | NUMERIC | SPECIAL) (ALPHA | NUMERIC | SPECIAL | "$")*

range_clause: "range" ID ":"? ID

ALPHA: /[a-zA-Z]/

NUMERIC: /[0-9]/

SPECIAL: "_" | "-" | "." | "~" | "%"
"""

In [ ]:
from lark import Lark

# Step 1: Read the grammar from a text file
with open("oml3_lark.txt", "r") as file:
    grammar = file.read()

# Step 2: Create the Lark parser using the grammar
parser = Lark(grammar)

# Step 3: Define a function to parse input
def parse_input(input_str):
    parse_tree = parser.parse(input_str)
    return parse_tree.pretty()

In [ ]:
# Step 4: Test the parser with an example input
input_str = """vocabulary <http://example.com/tutorial1/vocabulary/pizza#> as pizza {

    extends <http://www.w3.org/2001/XMLSchema#> as xsd

    extends <http://purl.org/dc/elements/1.1/> as dc

    extends <http://www.w3.org/2000/01/rdf-schema#> as rdfs


    @rdfs:comment "The class of things that are uniquely identified by id"
    aspect IdentifiedThing [
        key hasId
    ]

    @rdfs:comment "The id property of an identified thing"
    scalar property hasId [
        domain IdentifiedThing
        range xsd:string
        functional
    ]

    // Identified Things

    @rdfs:comment "The class of food items"
    concept Food < IdentifiedThing

    @rdfs:comment "A relation from a food to another used as an ingredient"
    relation entity HasIngredient [
        from Food
        to Food
        forward hasIngredient
        reverse isIngredientOf
        transitive
    ]

    @rdfs:comment "An enumeration of spiciness levels"
    scalar Spiciness [
        oneOf "Hot", "Medium", "Mild"
    ]

    @rdfs:comment "The spiciness property of a food item"
    scalar property hasSpiceness [
        domain Food
        range Spiciness
        functional
    ]

    // Foods

    @rdfs:comment "The class of pizzas"
    concept Pizza < Food [
        restricts some hasBase to PizzaBase
    ]

    @rdfs:comment "The class of pizza bases"
    concept PizzaBase < Food

    @rdfs:comment "The class of pizza toppings"
    concept PizzaTopping < Food

    @rdfs:comment "A relation from a pizza to a base"
    relation entity HasBase [
        from Pizza
        to PizzaBase
        forward hasBase
        reverse isBaseOf
        functional
        inverse functional
    ] < HasIngredient

    @rdfs:comment "A relation from a pizza to a topping"
    relation entity HasTopping [
        from Pizza
        to PizzaTopping
        forward hasTopping
        reverse isToppingOf
        inverse functional
    ] < HasIngredient

    // Pizzas

    @rdfs:comment "The class of pizzas with some cheese toppings"
    concept CheesyPizza < Pizza [
        restricts some hasTopping to CheeseTopping
    ]

    @rdfs:comment "The class of pizzas with some meat toppings"
    concept MeatyPizza < Pizza [
        restricts some hasTopping to MeatTopping
    ]

    @rdfs:comment "The class of pizzas with all vegetarian toppings"
    concept VegetarianPizza < Pizza [
        restricts all hasTopping to VegetarianTopping
    ]

    @rdfs:comment "The class of American pizzas"
    concept American < CheesyPizza, MeatyPizza [
        restricts some hasTopping to MozzarellaTopping
        restricts some hasTopping to SausageTopping
        restricts some hasTopping to TomatoTopping
    ]

    @rdfs:comment "The class of Veneziana pizzas"
    concept Veneziana < CheesyPizza, MeatyPizza [
        restricts some hasTopping to MozzarellaTopping
        restricts some hasTopping to TomatoTopping
        restricts some hasTopping to SultanaTopping
    ]

    @rdfs:comment "The class of Margherita pizzas"
    concept Margherita < CheesyPizza, VegetarianPizza [
        restricts some hasTopping to MozzarellaTopping
        restricts some hasTopping to TomatoTopping
    ]

    // Pizza Bases

    @rdfs:comment "The class of deep pan bases"
    concept DeepPanBase < PizzaBase

    @rdfs:comment "The class of thin and crispy bases"
    concept ThinAndCrispyBase < PizzaBase

    // Pizza Toppings

    @rdfs:comment "The class of meat toppings"
    concept MeatTopping < PizzaTopping

    @rdfs:comment "The class of vegetarian toppings"
    concept VegetarianTopping < PizzaTopping

    @rdfs:comment "The class of hot spiciness toppings"
    concept HotTopping < PizzaTopping [
        restricts hasSpiceness to "Hot"
    ]

    @rdfs:comment "The class of medium spiciness toppings"
    concept MediumTopping < PizzaTopping [
        restricts hasSpiceness to "Medium"
    ]

    @rdfs:comment "The class of mild spiciness toppings"
    concept MildTopping < PizzaTopping [
        restricts hasSpiceness to "Mild"
    ]

    // Meat Topping

    @rdfs:comment "The class sausage toppings"
    concept SausageTopping < MeatTopping, MildTopping
    @rdfs:comment "The class spiced beef toppings"
    concept SpicedBeefTopping < MeatTopping, HotTopping

    // Vegetarion Toppings

    @rdfs:comment "The class sauce toppings"
    concept SauceTopping < VegetarianTopping
    @rdfs:comment "The class cheese toppings"
    concept CheeseTopping < VegetarianTopping
    @rdfs:comment "The class fruit toppings"
    concept FruitTopping < VegetarianTopping
    @rdfs:comment "The class vegetable toppings"
    concept VegetableTopping < VegetarianTopping

    // Sauce Toppings

    @rdfs:comment "The class of tabasco toppings"
    concept TobascoTopping < SauceTopping, HotTopping

    // Cheese Toppings

    @rdfs:comment "The class of parmesan toppings"
    concept ParmesanTopping < CheeseTopping, MildTopping
    @rdfs:comment "The class of mozzarella toppings"
    concept MozzarellaTopping < CheeseTopping, MildTopping

    // Fruit Toppings

    @rdfs:comment "The class of sultana toppings"
    concept SultanaTopping < FruitTopping, MediumTopping

    // Vegetable Toppings

    @rdfs:comment "The class of pepper toppings"
    concept PepperTopping < VegetableTopping
    @rdfs:comment "The class of tomatoe toppings"
    concept TomatoTopping < VegetableTopping, MildTopping

    // Pepper Toppings

    @rdfs:comment "The class of jalapeno pepper toppings"
    concept JalapenoPepperTopping < PepperTopping, HotTopping
    @rdfs:comment "The class of sweet pepper toppings"
    concept SweetPepperTopping < PepperTopping, MildTopping
}
"""

# Parse the input and print the result
try:
    parsed_result = parse_input(input_str)
    print(parsed_result)
except Exception as e:
    print(f"Error: {e}")

start
  ontology
    vocabulary_box
      vocabulary
        <http://example.com/tutorial1/vocabulary/pizza#>
        pizza
        extension
          <http://www.w3.org/2001/XMLSchema#>
          xsd
        extension
          <http://purl.org/dc/elements/1.1/>
          dc
        extension
          <http://www.w3.org/2000/01/rdf-schema#>
          rdfs
        vocabulary_statement
          specializable_term
            type
              entity
                aspect
                  annotation
                    annotation_property_ref	rdfs:comment
                    annotation_value
                      literal
                        quoted_literal	"The class of things that are uniquely identified by id"
                  IdentifiedThing
                  key_axiom
                    property_ref	hasId
        vocabulary_statement
          specializable_term
            scalar_property
              annotation
                annotation_property_ref	rdfs:comment
      

In [ ]:
# Step 4: Test the parser with an example input
input_str = """@dc:creator "SIE Disruption Lab"
@dc:contributor "Joe Gregory"
@dc:description "A vocabulary to capture the foundation terminology required by the SIE Disruption Lab. Note that this is essentially a simplified version of gUFO."
@dc:hasVersion "0.1"
vocabulary <http://uaontologies.com/UA_Foundation/UA_Foundation#> as foundation {

	////////////////////////////////////////////////////////////////////////////////////

	/////// Vocabulary Imports ///////


	// Import standard vocabularies

    extends <http://purl.org/dc/elements/1.1/> as dc
    extends <http://www.w3.org/2000/01/rdf-schema#> as rdfs
    extends <http://www.w3.org/2001/XMLSchema#> as xsd


	///////////////////////////////////////////////////////////////////////////////////

	/////// Vocabulary Definitions ///////


	///// Aspects

	@rdfs:label "Contained Element"
	aspect ContainedElement

	@rdfs:label "Container"
	aspect Container

	@rdfs:label "SDC Carrier"
	aspect SDCCarrier

	@rdfs:label "GDC Carrier"
	aspect GDCCarrier


	///// Concepts

	@rdfs:label "IdentifiedEntity"
	concept IdentifiedEntity

	@rdfs:label "Occurrent"
	concept Occurrent < IdentifiedEntity, Container, ContainedElement, GDCCarrier, SDCCarrier [
		restricts all contains to Occurrent
		restricts all isContainedIn to Occurrent
	]

	@rdfs:label "Continuant"
	concept Continuant < IdentifiedEntity

	@rdfs:label "Temporal Region"
	concept TemporalRegion < Occurrent, Container, ContainedElement [
		restricts all contains to TemporalRegion
		restricts all isContainedIn to TemporalRegion
	]

	@rdfs:label "Process"
	concept Process < Occurrent, Container, ContainedElement, SDCCarrier [
		restricts all contains to Process
		restricts all isContainedIn to Process
	]

	@rdfs:label "Independent Continuant"
	concept IndependentContinuant < Continuant, SDCCarrier, GDCCarrier

	@rdfs:label "Specifically Dependent Continuant"
	concept SpecificallyDependentContinuant < Continuant, SDCCarrier, GDCCarrier [
		restricts some specificallyDependentOn to SDCCarrier
	]

	@rdfs:label "Generically Dependent Continuant"
	concept GenericallyDependentContinuant < Continuant

	@rdfs:label "Realizable Entity"
	concept RealizableEntity < SpecificallyDependentContinuant

	@rdfs:label "Quality"
	concept Quality < SpecificallyDependentContinuant

	@rdfs:label "Disposition"
	concept Disposition < RealizableEntity

	@rdfs:label "Function"
	concept Function < Disposition, Container, ContainedElement [
		restricts all contains to Function
		restricts all isContainedIn to Function
	]

	@rdfs:label "Material Entity"
	concept MaterialEntity < IndependentContinuant, ContainedElement

	@rdfs:label "Object Aggregate"
	concept ObjectAggregate < MaterialEntity, Container [
		restricts all contains to MaterialEntity
		restricts all isContainedIn to ObjectAggregate
	]

	@rdfs:label "Object"
	concept Object < MaterialEntity

	@rdfs:label "Immaterial Entity"
	concept ImmaterialEntity < IndependentContinuant

	@rdfs:label "Site"
	concept Site < ImmaterialEntity

	@rdfs:label "Energy"
	concept Energy < ImmaterialEntity

	@rdfs:label "Role"
	concept Role < GenericallyDependentContinuant, Container, ContainedElement, SDCCarrier [
		restricts all contains to Role
		restricts all isContainedIn to Role
	]



	///// Relations

	@rdfs:label "Contains"
	@dc:description "[=Contains=] is a..."
	relation entity Contains [
		from Container
		to ContainedElement
		@rdfs:label "contains"
		forward contains
		@rdfs:label "is contained in"
		reverse isContainedIn
		asymmetric
		irreflexive
	]

	@rdfs:label "Specifically Dependent On"
	@dc:description "[=SpecificallyDependentOn=] is a..."
	relation entity SpecificallyDependentOn [
		from SpecificallyDependentContinuant
		to SDCCarrier
		@rdfs:label "specifically dependent on"
		forward specificallyDependentOn
		@rdfs:label "has specific dependent"
		reverse hasSpecificDependent
		asymmetric
		irreflexive
	]

	@rdfs:label "Generically Dependent On"
	@dc:description "[=GenericallyDependentOn=] is a..."
	relation entity GenericallyDependentOn [
		from GenericallyDependentContinuant
		to GDCCarrier
		@rdfs:label "generically dependent on"
		forward genericallyDependentOn
		@rdfs:label "has generic dependent"
		reverse hasGenericDependent
		asymmetric
		irreflexive
	]

	@rdfs:label "Occupies"
	@dc:description "[=Occupies=] is a..."
	relation entity Occupies [
		from MaterialEntity
		to Site
		@rdfs:label "occupies"
		forward occupies
		@rdfs:label "is occupied by"
		reverse isOccupiedBy
		asymmetric
		irreflexive
	]



	///// Scalar Properties

	@rdfs:label "has name"
	@dc:description "[=hasName=] is a FOUNDATIONAL scalar property - all entities can have a name"
	scalar property hasName [
		domain IdentifiedEntity
		range xsd:string
	]

	@rdfs:label "has ID"
	@dc:description "[=hasID=] is a FOUNDATIONAL scalar property - all entities can have an ID"
	scalar property hasID [
		domain IdentifiedEntity
		range xsd:string
	]

	@rdfs:label "has natural language description"
	@dc:description "[=hasNaturalLanguageDescription=] is a FOUNDATIONAL scalar property - all entities can have a natural language description"
	scalar property hasNaturalLanguageDescription [
		domain IdentifiedEntity
		range xsd:string
	]

	@rdfs:label "has begin instant"
	@dc:description "[=hasBeginInstant=] ..."
	scalar property hasBeginInstant [
		domain TemporalRegion
		range xsd:dateTime
	]

	@rdfs:label "has end instant"
	@dc:description "[=hasEndInstant=] ..."
	scalar property hasEndInstant [
		domain TemporalRegion
		range xsd:dateTime
	]

	///// Rules
	}
"""

# Parse the input and print the result
try:
    parsed_result = parse_input(input_str)
    print(parsed_result)
except Exception as e:
    print(f"Error: {e}")

start
  ontology
    vocabulary_box
      vocabulary
        annotation
          annotation_property_ref	dc:creator
          annotation_value
            literal
              quoted_literal	"SIE Disruption Lab"
        annotation
          annotation_property_ref	dc:contributor
          annotation_value
            literal
              quoted_literal	"Joe Gregory"
        annotation
          annotation_property_ref	dc:description
          annotation_value
            literal
              quoted_literal	"A vocabulary to capture the foundation terminology required by the SIE Disruption Lab. Note that this is essentially a simplified version of gUFO."
        annotation
          annotation_property_ref	dc:hasVersion
          annotation_value
            literal
              quoted_literal	"0.1"
        <http://uaontologies.com/UA_Foundation/UA_Foundation#>
        foundation
        extension
          <http://purl.org/dc/elements/1.1/>
          dc
        extension
          

In [ ]:
# Step 4: Test the parser with an example input
input_str = """@dc:creator "SIE Disruption Lab"
@dc:contributor "Joe Gregory"
@dc:description "A vocabulary to capture the organization patterns required by the SIE Disruption Lab."
@dc:hasVersion "0.1"
vocabulary <http://uaontologies.com/UA_Core/UA_Agent#> as UA_Agent {

	////////////////////////////////////////////////////////////////////////////////////

	/////// Vocabulary Imports ///////


	// Import standard vocabularies

    extends <http://purl.org/dc/elements/1.1/> as dc
    extends <http://www.w3.org/2000/01/rdf-schema#> as rdfs
    //extends <http://www.w3.org/2001/XMLSchema#> as xsd

    // Import relevant vocabularies

    extends <http://uaontologies.com/UA_Foundation/UA_Foundation#> as foundation

	extends <http://uaontologies.com/UA_Core/UA_Information#> as info
	//extends <http://uaontologies.com/UA_Core/UA_Event#> as event


	///////////////////////////////////////////////////////////////////////////////////

	/////// Vocabulary Definitions ///////

	///// Aspects

	@rdfs:label "Producer"
	@dc:description "A [=Producer=] is a ..."
	aspect Producer

	@rdfs:label "Consumer"
	@dc:description "A [=Consumer=] is a ..."
	aspect Consumer

	@rdfs:label "Agent"
	@dc:description "An [=Agent=]..."
	aspect Agent < foundation:SDCCarrier



	///// Concepts

	@rdfs:label "Organization"
	@dc:description "A [=Organization=]..."
	concept Organization < Agent, foundation:MaterialEntity, foundation:Container, foundation:ContainedElement [
		restricts all foundation:contains to Organization
		restricts all foundation:isContainedIn to Organization
	]

	@rdfs:label "Person"
	@dc:description "A [=Person=]..."
	concept Person < Agent, foundation:MaterialEntity

	@rdfs:label "ProcessOwner"
	@dc:description "A [=ProcessOwner=] is a ..."
	concept ProcessOwner < foundation:Role

	@rdfs:label "Managed Process"
	@dc:description "A [=ManagedProcess=]..."
	concept ManagedProcess < foundation:Process

	@rdfs:label "Procedure"
	@dc:description "A [=Procedure=]..."
	concept Procedure < info:PrescriptiveInformationEntity [
		restricts all info:prescribes to foundation:Process
		restricts some info:prescribes to foundation:Process
	]

	@rdfs:label "Plan"
	@dc:description "A [=Plan=]..."
	concept Plan < info:PrescriptiveInformationEntity [
		restricts all info:prescribes to foundation:Process
		restricts some info:prescribes to foundation:Process
		restricts some foundation:contains to Procedure
	]



	///// Relations

	@rdfs:label "Belongs To"
	@dc:description "[=BelongsTo=] is a..."
	relation entity BelongsTo [
		from Person
		to Organization
		@rdfs:label "belongs to"
		forward belongsTo
		@rdfs:label "has belonging"
		reverse hasBelonging
		asymmetric
		irreflexive
	]

	@rdfs:label "Responsible For"
	@dc:description "[=ResponsibleFor=] is a..."
	relation entity ResponsibleFor [
		from ProcessOwner
		to ManagedProcess
		@rdfs:label "responsible for"
		forward responsibleFor
		@rdfs:label "is responsibility of"
		reverse isResponsibilityOf
		asymmetric
		irreflexive
	]

	@rdfs:label "Has Input"
	@dc:description "[=HasInput=] is a..."
	relation entity HasInput [
		from Consumer
		to foundation:Continuant
		@rdfs:label "has input"
		forward hasInput
		@rdfs:label "is input of"
		reverse isInputOf
		asymmetric
		irreflexive
	]

	@rdfs:label "Has Output"
	@dc:description "[=HasOutput=] is a..."
	relation entity HasOutput [
		from Producer
		to foundation:Continuant
		@rdfs:label "has output"
		forward hasOutput
		@rdfs:label "is output of"
		reverse isOutputOf
		asymmetric
		irreflexive
	]

	@rdfs:label "Is Role Of"
	@dc:description "[=IsRoleOf=] is a..."
	relation entity IsRoleOf [
		from foundation:Role
		to Agent
		@rdfs:label "is role of"
		forward isRoleOf
		@rdfs:label "has role"
		reverse hasRole
		asymmetric
		irreflexive
	]
	< foundation:GenericallyDependentOn

	@rdfs:label "Has Organizational Context"
	@dc:description "[=HasOrganizationContext=] is a "
	relation entity HasOrganizationalContext [
		from foundation:Role
		to Organization
		@rdfs:label "has organizational context"
		forward hasOrganziationalContext
		@rdfs:label "is organizational context of"
		reverse isOrganizationalContextOf
		asymmetric
		irreflexive
	]

	@rdfs:label "Has Planned Temporal Region"
	@dc:description "[=HasPlannedTemporalRegion=] is a "
	relation entity HasPlannedTemporalRegion [
		from Plan
		to foundation:TemporalRegion
		@rdfs:label "has planned temporal region"
		forward hasPlannedTemporalRegion
		@rdfs:label "is planned temporal region of"
		reverse isPlannedTemporalRegionOf
		asymmetric
		irreflexive
	]



	///// Scalar Properties



	///// Redefined Concepts

	ref concept foundation:Process < Producer, Consumer




	///// Rules


}
"""

# Parse the input and print the result
try:
    parsed_result = parse_input(input_str)
    print(parsed_result)
except Exception as e:
    print(f"Error: {e}")

start
  ontology
    vocabulary_box
      vocabulary
        annotation
          annotation_property_ref	dc:creator
          annotation_value
            literal
              quoted_literal	"SIE Disruption Lab"
        annotation
          annotation_property_ref	dc:contributor
          annotation_value
            literal
              quoted_literal	"Joe Gregory"
        annotation
          annotation_property_ref	dc:description
          annotation_value
            literal
              quoted_literal	"A vocabulary to capture the organization patterns required by the SIE Disruption Lab."
        annotation
          annotation_property_ref	dc:hasVersion
          annotation_value
            literal
              quoted_literal	"0.1"
        <http://uaontologies.com/UA_Core/UA_Agent#>
        UA_Agent
        extension
          <http://purl.org/dc/elements/1.1/>
          dc
        extension
          <http://www.w3.org/2000/01/rdf-schema#>
          rdfs
        extension
 

In [ ]:
# Function to visualize the parse tree
def visualize_parse_tree(parse_tree):
    def build_graph(tree, graph=None):
        if graph is None:
            graph = Digraph()
        if isinstance(tree, Tree):
            graph.node(tree.data)
            for child in tree.children:
                build_graph(child, graph)
                graph.edge(tree.data, str(child))
        else:
            # For Token, just use its value
            graph.node(str(tree))
        return graph

    graph = build_graph(parse_tree)
    return graph

# Visualize the parse tree
graph = visualize_parse_tree(parsed_result)
graph.render("pizza_tree", format="png", view=True)

'pizza_tree.png'